In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv

# dicというディレクトリにダウンロードしてきた極性辞書を入れておく
p_dic = pathlib.Path('dic')

# 極性辞書はテキスト形式でダウンロードしてきている想定
for i in p_dic.glob('*.dic'):
    with open (i, 'r', encoding = 'cp932') as f:
        #単語・読み仮名・品詞・スコアに分割してリストとして格納
        x = [ii.replace('\n', '').split(':') for ii in  f.readlines()]

posi_nega_df = pd.DataFrame(x, columns = ['基本形', '読み', '品詞', 'スコア'])
# jaconvを使って読み仮名を全てカタカナに変換
posi_nega_df['読み'] = posi_nega_df['読み'].apply(lambda x : jaconv.hira2kata(x))
# なぜか読みや品詞まで同じなのに、異なるスコアが割り当てられていたものがあったので重複を削除
posi_nega_df = posi_nega_df[~posi_nega_df[['基本形', '読み', '品詞']].duplicated()]
# print(posi_nega_df)


In [2]:
p_temp = pathlib.Path('text')

article_list = []

# フォルダ内のテキストファイルを全てサーチ
for p in p_temp.glob('**/*.txt'):
    #第二階層フォルダ名がニュースサイトの名前になっているので、それを取得
    media = p.parent.name
    file_name = p.name
    # print(file_name)

    if file_name != 'LICENSE.txt' and file_name != 'CHANGES.txt' and file_name != 'README.txt':
        #テキストファイルを読み込む
        with open(p, 'r') as f:
            #テキストファイルの中身を一行ずつ読み込み、リスト形式で格納
            article = f.readlines()
            #不要な改行等を置換処理
            article = [re.sub(r'[\n \u3000]', '', i) for i in article]
    #ニュースサイト名・記事URL・日付・記事タイトル・本文の並びでリスト化
            article_list.append([media, article[0], article[1], article[2], ''.join(article[3:])])
    else :
        continue


article_df = pd.DataFrame(article_list)

# article_df.head()
# print(article_df.shape) #(7376, 5)


In [3]:
news_df = article_df
# print(news_df)

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *

t = Tokenizer()
char_filters = [UnicodeNormalizeCharFilter()]
analyzer = Analyzer(char_filters=char_filters, tokenizer=t)

word_lists = []
for i, row in news_df.iterrows(): 
    # print(i) 
    # print(row[3])  
    # print(row[4])
    row_lists = []
    for t in analyzer.analyze(row[4]):
        #形態素
        surf = t.surface
        #基本形
        # base = t.base_form
        #品詞
        # pos = t.part_of_speech
        #読み
        # reading = t.reading
        
        row_lists.append(t)
    word_lists.append(row_lists)


# word_df = pd.DataFrame(word_lists, columns=['ニュース'])

# word_df = pd.DataFrame(word_lists, columns = ['ニュースNo.', '単語', '基本形', '品詞', '読み'])
# word_df['品詞'] = word_df['品詞'].apply(lambda x : x.split(',')[0])

In [4]:
article_list = []
for i in range(len(word_lists)):
    surface_words = [token.surface for token in word_lists[i]]
    joined = ' '.join(surface_words)
    article_list.append(joined)

df = pd.DataFrame(article_list, columns=["article"])

In [5]:
df.head()

,article
0,もうすぐ ジューン ・ ブライド と 呼ば れる 6 月 。 独 女 の 中 に は 自分 ...
1,携帯 電話 が 普及 する 以前 、 恋人 へ の 連絡 ツール は 一般 電話 が 普通 ...
2,「 男性 は やっぱり 、 女性 の “ すっぴん ” が 大好き な ん です か ね 」...
3,ヒップ の 加 齢 による 変化 は 「 たわむ → 下がる → 内 に 流れる 」 、 バ...
4,6 月 から 支給 さ れる 子ども 手当 だ が 、 当初 は 子ども 一 人 当たり 月...


In [6]:
df.to_csv('token.csv', encoding='utf-8')